# Class Decorators - Coding

In [8]:
def savings(cls):
    cls.account_type = "savings"
    return cls
    
def checking(cls):
    cls.account_type = "checking"
    return cls


In [14]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings 
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

In [15]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'savings'})

In [16]:
Bank2Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'checking'})

In [19]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [20]:
@account_type("Savings")
class Bank1Savings:
    pass

@account_type("Checking")
class Bank1Checking:
    pass

In [21]:
Bank1Savings.__dict__, Bank1Checking.__dict__

(mappingproxy({'__module__': '__main__',
               '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
               '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
               '__doc__': None,
               'account_type': 'Savings'}),
 mappingproxy({'__module__': '__main__',
               '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
               '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
               '__doc__': None,
               'account_type': 'Checking'}))

In [22]:
def hello(cls):
    cls.hello = lambda self: f"{self} says hello!"
    return cls

In [23]:
@hello
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name
    
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [25]:
p = Person("Guidop")
p.hello()

'Guidop says hello!'

In [26]:
from functools import wraps

In [56]:
def fun_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"Log: {fn.__qualname__}({args}, {kwargs} = {result})")
        return result
    return inner

In [29]:
class Person:
    @fun_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @fun_logger
    def great(self):
        return f"Hello, my name is {self.name}, and I am {self.age} years old"

In [30]:
p = Person("John", 78)

Log: Person.__init__((<__main__.Person object at 0x7f4dd03eb970>, 'John', 78), {} = None)


In [31]:
p.great()

Log: Person.great((<__main__.Person object at 0x7f4dd03eb970>,), {} = Hello, my name is John, and I am 78 years old)


'Hello, my name is John, and I am 78 years old'

In [35]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating", cls, name)
            setattr(cls, name, fun_logger(obj))
    return cls

In [36]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def great(self):
        return f"Hello, my name is {self.name}, and I am {self.age} years old"

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> great


In [34]:
p = Person("John", 78)

Log: Person.__init__((<__main__.Person object at 0x7f4db6e23910>, 'John', 78), {} = None)


In [37]:
p.great()

Log: Person.great((<__main__.Person object at 0x7f4db6e23910>,), {} = Hello, my name is John, and I am 78 years old)


'Hello, my name is John, and I am 78 years old'

In [38]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print("static_method invoked...")
        
    @classmethod
    def cls_method(cls):
        print(f"cls_method invoked for {cls}")
        
    def instance_method(self):
        print(f"instance_method invoked for {self}")

decorating <class '__main__.Person'> instance_method


In [39]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'static_method': <staticmethod at 0x7f4db6e02310>,
              'cls_method': <classmethod at 0x7f4db6e026d0>,
              'instance_method': <function __main__.Person.instance_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [41]:
callable(Person.__dict__["static_method"])

False

In [44]:
class Person:
    @fun_logger
    @staticmethod
    def static_method():
        pass

In [45]:
Person.static_method()

TypeError: 'staticmethod' object is not callable

In [46]:
class Person:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def class_method(cls):
        pass

In [48]:
type(Person.__dict__["static_method"])

staticmethod

In [47]:
type(Person.__dict__["class_method"])

classmethod

In [50]:
Person.__dict__["static_method"].__func__

<function __main__.Person.static_method()>

In [57]:
def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"Log: {fn.__qualname__}({args}, {kwargs} = {result})")
        return result
    return inner

In [58]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print("decorating static method")
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print("decorating classmethod method")
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)            
    return cls       

In [59]:
@class_logger
class Person:
    @staticmethod
    def static_method(a, b):
        print("static_method called ...", a, b)
    
    @classmethod
    def class_method(cls, a, b):
        print("class method called ...", a, b)
        
    def instance_method(self, a, b):
        print("instance method called ...", a, b)

decorating static method
decorating classmethod method
decorating callable <class '__main__.Person'> instance_method


In [65]:
Person.static_method(10, 20)

static_method called ... 10 20
Log: Person.static_method((10, 20), {} = None)


In [66]:
p = Person()
p.instance_method(10, 20)

instance method called ... 10 20
Log: Person.instance_method((<__main__.Person object at 0x7f4db671e460>, 10, 20), {} = None)


In [67]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating callable <class '__main__.Person'> __init__


In [68]:
type(vars(Person)['name'])

property

In [69]:
isinstance(Person.__dict__["name"], property)

True

In [71]:
prop = Person.__dict__["name"]
prop.fset, prop.fget, prop.fdel

(None, <function __main__.Person.name(self)>, None)

In [73]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print("decorating static method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print("decorating classmethod method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)  
        elif isinstance(obj, property):
            print("decorating property", cls, name)
            if obj.fget is not None:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset is not None:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel is not None:
                obj = obj.deleter(func_logger(obj.fdel))    
            setattr(cls, name, obj)    
    return cls       

In [74]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    

decorating callable <class '__main__.Person'> __init__
decorating property <class '__main__.Person'> name


In [76]:
p = Person("David")

Log: Person.__init__((<__main__.Person object at 0x7f4db6b2c430>, 'David'), {} = None)


In [77]:
p.name

Log: Person.name((<__main__.Person object at 0x7f4db6b2c430>,), {} = David)


'David'

In [79]:
@class_logger
class Person:
    class Other:
        def __call__(self):
            print("called instance of other...")
    other = Other()

decorating callable <class '__main__.Person'> Other
decorating callable <class '__main__.Person'> other


In [80]:
Person.Other

<function __main__.Person.Other()>

In [81]:
Person.other

<function __main__.func_logger.<locals>.inner()>

In [82]:
import inspect

In [83]:
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()

In [84]:
keys = ("static_method", "cls_method", "inst_method", 
        "name", "__add__", "Other", "other")
inspect_funcs = ("isroutine", "ismethod", "isfunction",
                "isbuiltin", "ismethoddescriptor")

In [86]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


In [88]:
def class_logger(cls):
    for name, obj in vars(cls).items():

        if isinstance(obj, staticmethod):
            print("decorating static method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print("decorating classmethod method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)  
        elif isinstance(obj, property):
            print("decorating property", cls, name)
            if obj.fget is not None:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset is not None:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel is not None:
                obj = obj.deleter(func_logger(obj.fdel))    
            setattr(cls, name, obj)    
        elif inspect.isroutine(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls       

In [89]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()

decorating static method <class '__main__.MyClass'> static_method
decorating classmethod method <class '__main__.MyClass'> cls_method
decorating callable <class '__main__.MyClass'> inst_method
decorating property <class '__main__.MyClass'> name
decorating callable <class '__main__.MyClass'> __add__


In [3]:
list(reversed(range(11,1,-1)))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [94]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, classmethod) or isinstance(obj, staticmethod):
            print(f"decorating: {name}")
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)            

        elif isinstance(obj, property):
            print("decorating property", cls, name)
            if obj.fget is not None:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset is not None:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel is not None:
                obj = obj.deleter(func_logger(obj.fdel))    
            setattr(cls, name, obj)    
        elif inspect.isroutine(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls       

In [95]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()

decorating: static_method
decorating: cls_method
decorating callable <class '__main__.MyClass'> inst_method
decorating property <class '__main__.MyClass'> name
decorating callable <class '__main__.MyClass'> __add__


In [98]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, classmethod) or isinstance(obj, staticmethod):
            print(f"decorating: {name}")
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)            

        elif isinstance(obj, property):
            print("decorating property", cls, name)
            methods = {("fget", "getter"), ("fset", "setter"),
                      ("fdel", "deleter")}
            for prop, method in methods:
                if getattr(obj, prop):
                    obj = getattr(obj, method)(func_logger(getattr(obj, prop)))
            setattr(cls, name, obj)    
        elif inspect.isroutine(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls       

In [99]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()

decorating: static_method
decorating: cls_method
decorating callable <class '__main__.MyClass'> inst_method
decorating property <class '__main__.MyClass'> name
decorating callable <class '__main__.MyClass'> __add__
